In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import numpy as np
import pandas as pd
import plotly.express as px
from scipy.fft import fft,ifft
import matplotlib.pyplot as plt
import os

In [2]:
#getting experiment list
experiments = os.listdir("Person_A")
experiments.sort()
for i in range(len(experiments)):
    if(i==0):experiments[i] = experiments[i][:-4]
    else: experiments[i] = experiments[i][:-5]
experiments = set(experiments)
experiments = list(dict.fromkeys(experiments))
print(experiments)
type(experiments)

['flex_medium', 'electrode_flex_thumbref', 'flex_weak', 'electrode_flex_paper', 'electrode_flex_tableref', 'electrode_per_finger', 'Wrist_flex', 'flex_fingers', 'electrode_flex_headref', 'flex_strong', 'Reference', 'electrode_flex_chestref', 'electrode_flex_forearmref']


list

In [23]:
#loading data
def load(person = "C",exp="Reference"):
    return pd.read_csv("Person_"+person+"/"+exp+".csv",header=None,names=["vals"])

#rectify signal
def rectify(data):
    data.vals = np.absolute(data.vals)
    return data
def rec_mean(data):
    data.vals = data.vals-data.vals.mean()
    return data

def fft_filter(data):
    fft = np.fft.fft(data.vals)
    to_filter = np.arange(950,1050)
    fft[1250+to_filter].real = 0
    fft[to_filter-1250].real = 0
    data.vals = np.fft.ifft(fft)
    return data
def moving_average(data,window_size):
    data.vals = data.vals.rolling(window_size).mean()
    return data
def plot(fig,axs,data,p,stdeb=False):
    xs = np.arange(0,len(data),1)/500
    std = data.rolling(75).std()
    axs[p].plot(xs,data,alpha=0.4)
    if(stdeb): axs[p].fill_between(xs, data.vals-std.vals, data.vals+std.vals,alpha=0.2)

In [24]:
def graph(person,exp,rc=True,rm=True,ffilt=True,ma=True,stdeb=True):
    ps = ["A","B","C"]
    ref = load(person,"Reference") #reference signal for person
    fig, axs = plt.subplots(3, 1,figsize=(10,8))

    for p in range(len(ps)):
        if(exp == "Reference"): 
            ffilt = False #if looking at reference signal, can't filter by itself
            sig = load(ps[p],exp) #signal of interest
            if(rm): sig = rec_mean(sig)
            if(rc): sig = rectify(sig)
            if(ffilt): sig = fft_filter(sig)
            if(ma!=0): sig = moving_average(sig,ma)
            plot(fig,axs,sig,p,stdeb)
        else:
            for i in range(1,4):
                sig = load(ps[p],exp+str(i)) #signal of interest
                if(rm): sig = rec_mean(sig)
                if(rc): sig = rectify(sig)
                if(ffilt): sig = fft_filter(sig)
                if(ma!=0): sig = moving_average(sig,ma)
                plot(fig,axs,sig,p,stdeb)


In [25]:
ref = load("A")
sig = load("A","flex_fingers1")
%matplotlib inline  
interact(graph, person=["A","B","C"],exp=experiments,rc=True,ffilt=False,ma=75,stdeb=True);

interactive(children=(Dropdown(description='person', options=('A', 'B', 'C'), value='A'), Dropdown(description…